# Processament de llenguatge natural amb models pre-entrenats

En l'exemple anterior hem vist com utilitzar llibreries com Nltk i TextBlob per processar text. Encara que son eficients i fàcils d'utilitzar, no sempre són les millors opcions per processar text en llenguatges diferents a l'anglès.

Els models pre-entrenats són models que han estat entrenats amb grans quantitats de dades i que poden ser utilitzats per processar text en diferents idiomes. Aquests models requereixen més dades i temps de computació per ser entrenats, però una vegada entrenats si son eficients.

El seu entrenament també és menys manual i facilita la seva utilització en diferents idiomes.

En aquesta pràctica veurem com utilitzar models pre-entrenats per processar text en diferents idiomes.

Utilitzarem els models pre-entrenats de la llibreria `spaCy` pero podriem utilitzar altres coleccions de models com `flair`.

## Inicialització de l'entorn

En primer lloc haurem d'instal·lar la llibreria `spaCy` i els models pre-entrenats en diferents idiomes; en aquest cas instal·larem el model de llenguatge en català `ca_core_news_sm`.

In [88]:
!pip install spacy


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [19]:
import spacy

spacy.cli.download("ca_core_news_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 19.0 MB/s eta 0:00:0000:0100:01



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


✔ Download and installation successful
You can now load the package via spacy.load('ca_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


A continuació carregarem el model.

In [20]:
nlp = spacy.load("ca_core_news_sm")

## Tokenització

Per tokenitzar amb spacy simplement cridem el mètode `nlp` amb el text que volem processar. Aquest mètode retorna un objecte `Doc` que farem servir per gran part de les operacions.

Per accedir als tokens podem recorrer l'objecte `Doc` com si fos una llista.

In [94]:
doc = nlp("Els dilluns a la tarda són molt llargs.")
print([token.text for token in doc])

['Els', 'dilluns', 'a', 'la', 'tarda', 'són', 'molt', 'llargs', '.']


Una de les funcions més interessants d'Spacy és que ens permet obtenir la categoria gramatical de cada paraula. Per fer-ho, simplement accedim a l'atribut `pos_` de cada paraula.

In [93]:
for token in doc:
    print(token.text, token.pos_)

Els DET
dilluns NOUN
a ADP
la DET
tarda NOUN
són AUX
molt ADV
llargs ADJ
. PUNCT
esquerra NOUN


Les categories gramaticals es poden consultar a: https://universaldependencies.org/u/pos/ o rebre una descripció amb `spacy.explain`.

In [40]:
for token in doc:
    print(token.text, token.pos_, spacy.explain(token.pos_))

El DET determiner
Barça PROPN proper noun
és AUX auxiliary
el DET determiner
millor ADJ adjective
equip NOUN noun
d ADP adposition
el DET determiner
món NOUN noun
. PUNCT punctuation
De ADP adposition
vegades NOUN noun
. PUNCT punctuation


### Tokenització de frases

En alguns casos, també és necessari tokenitzar el text en frases. Per fer-ho, utilitzarem el métode `sents` de l'objecte `Doc`.

In [30]:
# Tokenitzem el text en frases

text = "El Barça és el millor equip del món. De vegades."
doc = nlp(text)

for sent in doc.sents:
    print(sent.text)

El Barça és el millor equip del món.
De vegades.


## Stopwords i signes de puntuació

Les *stopwords* són paraules que no aporten informació rellevant per a la tasca que estem realitzant. Per exemple, en la tasca de classificació de text, les *stopwords* no aporten informació per a la classificació.

Per tant, en molts casos, és recomanable eliminar les *stopwords* del text abans d'aplicar qualsevol tècnica de processament de llenguatge natural.

TextBlob ens permet eliminar les *stopwords* del text. Per fer-ho, utilitzarem la propietat de *is_stop* de cadascún dels tokens del **Doc**.

A continuació, veurem un exemple de com eliminar les *stopwords* d'un text.

In [34]:
# Eliminem les stopwords d'un text

text = "El Barça és el millor equip del món. De vegades."
doc = nlp(text)
tokens = [token for token in doc if not token.is_stop]

tokens

[Barça, millor, equip, d, món, ., vegades, .]

També podem veure com, junt als stopwords, també podem llevar els signes de puntuació.

In [36]:
tokens = [token for token in doc if not token.is_stop and not token.is_punct]
tokens

[Barça, millor, equip, d, món, vegades]

## Lematització i stemming

La lematització és el procés de convertir una paraula a la seva forma base. Per exemple, la paraula *està* es converteix en *estar*. L'stemming, en canvi, consisteix en eliminar els afixos de les paraules. Per exemple, la paraula *està* es converteix en *est*.

Ambdues tècniques ens permeten reduir el vocabulari del text i, per tant, reduir la dimensionalitat dels vectors de paraules; cosa que ens pot ajudar a millorar el rendiment dels nostres models.

Spacy no inclou un mètode per fer stemming, però si que inclou un mètode per fer lematització. Per fer-ho, accedim a l'atribut `lemma_` de cada paraula.

La raó per la qual Spacy no inclou un mètode per fer stemming és perquè la lematització és més precisa que el stemming i spacy està dissenyat per a tasques de producció on la precisió és més important que la velocitat.

In [46]:
# Lematitzem i stemitzem un text

text = "Series un bon cantant, si no fora per la veu."
doc = nlp(text)

# Lematitzem
lemmas = [token.lemma_ for token in doc]

lemmas

['seria',
 'un',
 'bo',
 'cantant',
 ',',
 'si',
 'no',
 'fora',
 'per',
 'el',
 'veu',
 '.']

## Representació del text

Un cop hem pre-processat el text, hem de representar-lo en un format que pugui ser entès per l'ordinador. 

Com ja hem comentat Spacy està centrat en tasques de producció i, per tant inclou els métodes més comuns actualment; especialment els basats en vectors de característiques.

### Embeddings

Spacy inclou un mètode per obtenir els embeddings del document i de la paraula. Aquests embeddings són vectors de característiques que representen el document o la paraula.

Els embeddings són útils per a tasques de classificació de text, agrupació de text, etc.

Com estem utilitzant el model petit (`ca_core_news_sm`), els embeddings son menys precisos i solament a nivell de document i no a nivell de paraula. Així i tot, ens poden ser útils per a tasques senzilles.

Si necessitem embeddings més precisos, podem utilitzar els models mitjà (`ca_core_news_md`) o gran (`ca_core_news_lg`).

In [47]:
doc.vector

array([ 1.7305475 , -0.12840228, -0.5396843 , -0.83717424, -1.3100742 ,
        0.17650956, -0.1620681 ,  0.48682687,  0.06764957,  0.18184917,
        0.8215378 , -0.40531054,  0.25735107,  0.93934184, -0.06517167,
        0.04939266, -0.4045429 ,  0.24763483,  1.0905582 ,  0.6970573 ,
       -0.31526443,  1.4649282 , -0.23816238, -0.4865251 , -0.09747415,
       -0.28767216,  0.5814107 , -0.7125825 ,  0.49832496,  0.8188111 ,
       -0.15080123,  0.1994148 ,  0.2723398 ,  0.8346703 , -0.7402689 ,
       -1.4915727 , -0.9905634 ,  0.42298186,  0.6075589 ,  0.29053253,
       -0.00331459, -1.3474356 , -0.58441734, -0.7741163 ,  0.05676066,
       -0.17020352,  0.40141544,  0.6308312 , -0.40938064,  0.6958816 ,
        0.3174107 ,  1.168798  , -0.04763202,  0.19581653,  0.08472364,
       -0.01598583,  0.00706673, -0.8146202 , -0.65326816, -0.6593003 ,
       -1.0186236 , -1.1242762 ,  0.1646251 ,  0.07394195, -0.7488492 ,
       -0.5852267 , -0.8377841 , -0.47709122,  0.6825135 ,  0.55

## Similaritat de text

Aprofitant els embeddings, Spacy ens permet calcular la similitud entre dos documents o dos paraules. Aquesta similitud es calcula utilitzant la similitud del cosinus entre els vectors de característiques.

Podem aprofiar aquesta funcionalitat per a recomanadors, cercadors de text, etc.

Ens caldrà carregar el model mitjà (`ca_core_news_md`) o gran (`ca_core_news_lg`) per a poder calcular el vector de característiques de les paraules i per buscar similituds entre vectors.

### Exemple

In [65]:
# Carreguem un model de paraules

spacy.cli.download("ca_core_news_md")
nlp = spacy.load("ca_core_news_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 11.7 MB/s eta 0:00:00a 0:00:01



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


✔ Download and installation successful
You can now load the package via spacy.load('ca_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [67]:
# Creem dues frases i calculem la similitud

text1 = "M'agrada el futbol."

text2 = "M'agrada el bàsquet."

doc1 = nlp(text1)
doc2 = nlp(text2)

doc1.similarity(doc2)

0.9890047051139093

### Sinónims

Podriem utilitzar la similitud de text per trobar sinónims. Per exemple, podem calcular la similitud entre dues paraules i si la similitud és superior a un cert llindar, podem considerar que són sinónims.

De tota manera, aquesta tècnica no és perfecta i no sempre funciona. Per exemple, la paraula *cotxe* i *vehicle* són sinònims, però la similitud entre elles pot no ser suficientment alta.

En aquests casos, podem utilitzar un tesaure com WordNet per trobar sinònims. Utilitzarem `spacy-wordnet` per integrar WordNet amb Spacy.

In [78]:
# Utilitzem wordnet i omw per obtenir sinònims

!pip install nltk


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [85]:
# Descarreguem els recursos necessaris

import nltk

nltk.download("wordnet")
nltk.download("omw")
nltk.download("omw-1.4")

[nltk_data] Downloading package wordnet to /home/carles/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw to /home/carles/nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/carles/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [87]:
# Obtenim els sinònims de la paraula 'gos'

from nltk.corpus import wordnet as wn

for synset in wn.synsets(u'gos',lang=('cat')):
    for lemma in synset.lemma_names(u'cat'):
        print(lemma)

ca
canis_familiaris
gos
gos_domèstic


# Conclusions

En aquesta pràctica hem vist com pre-processar text i com representar-lo en un format que pugui ser entès per l'ordinador. A més, hem vist com utilitzar algunes de les funcionalitats de Spacy i llibreries relacionades.

Spacy és una llibreria molt potent i fàcil d'utilitzar per processar text en diferents idiomes. A més, inclou moltes funcionalitats útils per a tasques de processament de llenguatge natural.